In [ ]:
# Earnings (today → +3 days) – IV / HV • Yahoo Screener • multithread • Jupyter
# ---------------------------------------------------------------------------------
# Kullanım:
# 1. Aşağıdaki **Gerekli Paketler** listesini kendi ortamınıza _manuel_ yükleyin.
#    Sandbox veya kısıtlı ortamlarda `subprocess` kullanılamadığından kod
#    otomatik `pip install` yapmaz.
# 2. Bu hücreyi Jupyter Notebook / JupyterLab içine *tek parça* yapıştırıp
#    çalıştırın. Çalışma sürecini durdurmak için hücreyi «Interrupt/Stop» edin.
#
# Gerekli Paketler
# ----------------
#   yfinance>=0.2.38
#   pandas>=1.4
#   numpy>=1.23
#   requests>=2.28
#   beautifulsoup4>=4.12
#   lxml>=4.9
#   tqdm>=4.66  (ipywidgets>=7 önerilir – yüklü değilse metin moduna döner)
#   websockets>=11  (yfinance bağımlılığı)
#
# Önemli Notlar
# -------------
# • Kod _otomatik kurulum_ yapmaz; eksik modül varsa temiz bir ImportError verir.
# • Eğer ipywidgets yoksa `tqdm` progress bar'ı metin modunda görünür.
# • `os.environ["YF_NO_CURL"] = "1"` ile yfinance'ın `curl_cffi` yolunu
#   kapatıyoruz, böylece Ctrl‑C/Interrupt sorunsuz çalışıyor.

import os, time, logging, concurrent.futures, requests
from datetime import date, timedelta
from math import sqrt

# --- 3.‑parti modüller -------------------------------------------------------
try:
    import yfinance as yf
    import pandas as pd
    import numpy as np
    from bs4 import BeautifulSoup
    from IPython.display import clear_output, display
    from tqdm.auto import tqdm
except ImportError as e:
    missing = str(e).split("'")[1]
    raise ImportError(
        f"Gerekli paket '{missing}' yüklü değil. Lütfen yukarıdaki listeye göre pip install yapın.")

# --- Genel ayarlar -----------------------------------------------------------
os.environ["YF_NO_CURL"] = "1"              # Ctrl‑C uyumluluğu
logging.getLogger("yfinance").setLevel(logging.ERROR)
TODAY       = date.today()
LIMIT_DATE  = TODAY + timedelta(days=3)
REFRESH_SEC = 60
THREADS     = min(32, (os.cpu_count() or 4) * 2)

# --- 1. Opsiyonlu sembolleri çek (Yahoo Screener) ---------------------------

def get_optionable_symbols(max_pages: int = 120, pause: float = 0.4):
    """Yahoo Screener 'optionable' listesini getirir (≈ 4 000 sembol)."""
    url_tpl = (
        "https://query2.finance.yahoo.com/v1/finance/screener/predefined/saved"
        "?scrIds=optionable&count=100&start={}"
    )
    headers = {"User-Agent": "Mozilla/5.0"}
    symbols = []

    for page in range(max_pages):
        start = page * 100
        resp = requests.get(url_tpl.format(start), headers=headers, timeout=10)
        # JSON bekleniyor; değilse sayfayı atla
        if not resp.headers.get("Content-Type", "").startswith("application/json"):
            time.sleep(pause)
            continue
        try:
            js = resp.json() or {}
            quotes = ((js.get("finance") or {})
                      .get("result", [{}])[0]
                      .get("quotes", []))
            if not quotes:
                break  # sayfa bitti
            symbols.extend(q.get("symbol") for q in quotes if q.get("symbol"))
        except (ValueError, TypeError):
            pass  # bozuk JSON – sayfayı atla
        time.sleep(pause)  # rate‑limit
    return symbols

OPTIONABLE = get_optionable_symbols()
print(f"Optionable universe size: {len(OPTIONABLE)}")

# --- 2. Earnings tarama (paralel) -------------------------------------------

def next_earn_ts(sym: str):
    t = yf.Ticker(sym)
    try:
        if hasattr(t, "get_next_earnings_date"):
            ts = t.get_next_earnings_date()
            if pd.notna(ts):
                return sym, ts
        ed = t.get_earnings_dates(limit=6)
        fut = ed[ed.index.date >= TODAY]
        return (sym, fut.index[0]) if not fut.empty else (sym, None)
    except Exception:
        return sym, None


def earnings_window(symbols):
    keep: dict[str, pd.Timestamp] = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=THREADS) as ex:
        futures = {ex.submit(next_earn_ts, s): s for s in symbols}
        for f in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="earnings scan"):
            sym, ts = f.result()
            if ts is not None and TODAY <= ts.date() <= LIMIT_DATE:
                keep[sym] = ts
    return keep

# --- 3. IV / HV hesapları ----------------------------------------------------

def atm_iv(sym: str):
    try:
        t = yf.Ticker(sym)
        exp  = sorted(t.options)[0]
        oc   = t.option_chain(exp)
        spot = t.fast_info.get("lastPrice") or t.history(period="1d")["Close"].iloc[-1]
        calls = oc.calls.assign(d=lambda d: abs(d.strike - spot))
        puts  = oc.puts .assign(d=lambda d: abs(d.strike - spot))
        civ   = calls.sort_values("d").iloc[0]["impliedVolatility"]
        piv   = puts .sort_values("d").iloc[0]["impliedVolatility"]
        if pd.isna(civ) or pd.isna(piv):
            return None
        return (civ + piv) / 2
    except Exception:
        return None


def hist_vol(sym: str, win: int = 30):
    try:
        px = yf.download(sym, period=f"{win*2}d", progress=False)["Close"].tail(win + 1)
        if len(px) < win + 1:
            return None
        return np.log(px / px.shift(1)).dropna().std(ddof=0) * sqrt(252)
    except Exception:
        return None

# --- 4. Canlı döngü ---------------------------------------------------------
try:
    while True:
        clear_output(wait=True)
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}]  Earnings (today → +3d)  –  IV / HV\n")

        upcoming = earnings_window(OPTIONABLE)  # {sym: ts}

        records = []
        for sym, ts in tqdm(upcoming.items(), desc="IV/HV calc"):
            iv = atm_iv(sym)
            hv = hist_vol(sym)
            if iv is None or hv in (None, 0):
                continue
            records.append({
                "Earnings": ts.strftime("%d-%m-%Y %H:%M"),
                "Ticker"  : sym,
                "30d IV"  : f"{iv*100:5.1f}%",
                "30d HV"  : f"{hv*100:5.1f}%",
                "IV/HV"   : f"{iv/hv:4.2f}",
            })

        cols = ["Earnings", "Ticker", "30d IV", "30d HV", "IV/HV"]
        df = pd.DataFrame(records, columns=cols)
        if not df.empty:
            df = df.sort_values("Earnings").reset_index(drop=True)
        display(df if not df.empty else pd.DataFrame({"info": ["No optionable earnings in window."]}))

        time.sleep(REFRESH_SEC)
except KeyboardInterrupt:  # hücre «Interrupt» edildiğinde
    clear_output()
    print("Stopped.")


[2025-05-14 10:24:04]  Earnings (today → +3d)  –  IV / HV



earnings scan: 0it [00:00, ?it/s]
IV/HV calc: 0it [00:00, ?it/s]


,info
0,No optionable earnings in window.


In [ ]:
# -- Optionable symbol list (Yahoo Screener) ---------------------------------
# Gereken paketler: requests, pandas
import requests, pandas as pd, time, json

def get_optionable_symbols(max_pages: int = 120, pause: float = 0.3) -> list[str]:
    """
    Yahoo screener 'optionable' sonuçlarını 100'lük sayfalarda indirir.
    max_pages * 100 kayda kadar tarar, JSON gelmeyen sayfaları atlar.
    """
    base = ("https://query2.finance.yahoo.com/v1/finance/screener/predefined/saved"
            "?scrIds=optionable&count=100&start={}")
    hdrs = {"User-Agent": "Mozilla/5.0"}
    
    symbols: list[str] = []
    for page in range(max_pages):
        start = page * 100
        resp = requests.get(base.format(start), headers=hdrs, timeout=10)

        if resp.headers.get("Content-Type", "").startswith("application/json"):
            try:
                js = resp.json()
                quotes = (
                    js.get("finance", {})
                      .get("result", [{}])[0]
                      .get("quotes", [])
                )
                # Sayfa boşsa döngüyü bitir
                if not quotes:
                    break
                symbols.extend(q["symbol"] for q in quotes)
            except (ValueError, KeyError, TypeError):
                print(f"[warn] JSON parse failed for start={start}, sayfa atlandı")
        else:
            print(f"[warn] Non-JSON response (start={start}), sayfa atlandı")
        
        time.sleep(pause)                # rate-limit koruması
    
    return symbols

# ---- Hücre çıktısı ----------------------------------------------------------
symbols = get_optionable_symbols()
print(f"Toplam optionable sembol sayısı: {len(symbols)}")
print("İlk 25 sembol:", symbols[:25])


[warn] JSON parse failed for start=0, sayfa atlandı
[warn] JSON parse failed for start=100, sayfa atlandı
[warn] JSON parse failed for start=200, sayfa atlandı
[warn] JSON parse failed for start=300, sayfa atlandı
[warn] JSON parse failed for start=400, sayfa atlandı
[warn] JSON parse failed for start=500, sayfa atlandı
[warn] JSON parse failed for start=600, sayfa atlandı
[warn] JSON parse failed for start=700, sayfa atlandı
[warn] JSON parse failed for start=800, sayfa atlandı
[warn] JSON parse failed for start=900, sayfa atlandı
[warn] JSON parse failed for start=1000, sayfa atlandı


KeyboardInterrupt: 

In [ ]:
# Earnings Dashboard (today → +7 d) – IV / HV  •  robust network version
# ---------------------------------------------------------------------
# REQUIRED pip packages (no auto-install in notebook!):
#   yfinance>=0.2.41  pandas  numpy  requests  tqdm  websockets   (ipywidgets optional)

import os, time, concurrent.futures, logging, re
import requests, yfinance as yf
import pandas as pd, numpy as np
from datetime import date, timedelta
from math import sqrt
from IPython.display import clear_output, display
from tqdm.auto import tqdm   # text fallback if ipywidgets is absent

# ── ENV / Logging ────────────────────────────────────────────────────────────
os.environ["YF_NO_CURL"] = "true"          # disable curl_cffi completely
os.environ["YF_USE_CURL_CFFI"] = "false"   # secondary guard
logging.getLogger("yfinance").setLevel(logging.ERROR)

# ── USER SETTINGS ────────────────────────────────────────────────────────────
CSV_PATH  = r"C:\Users\PC\Desktop\Dokümanlar\Sistem Test\Earnings Plays\symbols.csv"
THREADS   = 32
REFRESH   = 120        # sec
TODAY     = date.today()
LIMIT     = TODAY + timedelta(days=7)
RETRIES   = 3         # per request

# ── Helpers ──────────────────────────────────────────────────────────────────
HDRS = {"User-Agent": "Mozilla/5.0"}

def safe_request(url, params=None):
    delay = 0.5
    for _ in range(RETRIES):
        try:
            r = requests.get(url, headers=HDRS, params=params, timeout=6)
            if r.ok and r.headers.get("Content-Type","").startswith("application/json"):
                return r.json()
        except requests.RequestException:
            pass
        time.sleep(delay); delay *= 2
    return None                      # give up

def sanitize(sym: str) -> str:
    return sym.strip().lstrip("$").upper().replace("/", "-").replace(".", "-")

def next_earn_ts(tkr: yf.Ticker):
    try:
        if hasattr(tkr, "get_next_earnings_date"):
            ts = tkr.get_next_earnings_date()
            if pd.notna(ts):
                return ts
        ed = tkr.get_earnings_dates(limit=6)
        fut = ed[ed.index.date >= TODAY]
        return fut.index[0] if not fut.empty else None
    except Exception:
        return None

def atm_iv(sym):
    try:
        t = yf.Ticker(sym)
        if not t.options: return None
        exp  = sorted(t.options)[0]
        oc   = t.option_chain(exp)
        spot = t.fast_info["lastPrice"]
        calls = oc.calls.assign(d=lambda d: abs(d.strike-spot))
        puts  = oc.puts .assign(d=lambda d: abs(d.strike-spot))
        civ, piv = (calls.sort_values("d").iloc[0]["impliedVolatility"],
                    puts .sort_values("d").iloc[0]["impliedVolatility"])
        if pd.isna(civ) or pd.isna(piv): return None
        return (civ + piv)/2
    except Exception:
        return None

def hist_vol(sym, win=30):
    try:
        px = yf.download(sym, period=f"{win*2}d", progress=False)["Close"].tail(win+1)
        if len(px) < win+1: return None
        return np.log(px/px.shift(1)).dropna().std(ddof=0)*sqrt(252)
    except Exception:
        return None

# ── Main loop ────────────────────────────────────────────────────────────────
COLS = ["Ticker","Company name","Earnings Date","IV/HV ratio"]

try:

        # 0) Yahoo crumb + cookie almak için tek seferlik warm-up -------------
    try:
        yf.Ticker("SPY").history(period="1d")
    except Exception as e:
        print("[warn] warm-up cookie fetch failed:", type(e).__name__, e)

    while True:
        clear_output(wait=True)
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}]  "
              "Earnings Dashboard (today → +7 d) – IV/HV\n")

        tick_raw = pd.read_csv(CSV_PATH, header=None)[0].astype(str).tolist()
        symbols  = [sanitize(s) for s in tick_raw]

        non_opt, records = [], []

        def worker(sym):
            tkr = yf.Ticker(sym)
            if not tkr.options:
                return ("no_opt", sym)
            ts = next_earn_ts(tkr)
            if ts and TODAY <= ts.date() <= LIMIT:
                return ("keep", sym, ts, tkr)
            return ("skip", sym)

        with concurrent.futures.ThreadPoolExecutor(max_workers=THREADS) as ex:
            jobs = [ex.submit(worker, s) for s in symbols]
            for fut in tqdm(concurrent.futures.as_completed(jobs),
                            total=len(jobs), desc="earnings scan"):
                res = fut.result()
                if res[0]=="no_opt": non_opt.append(res[1])
                elif res[0]=="keep":
                    sym, ts, tkr = res[1:]
                    iv, hv = atm_iv(sym), hist_vol(sym)
                    if iv is None or hv in (None,0): continue
                    name = (tkr.fast_info.get("shortName") or
                            tkr.info.get("shortName","—"))[:40]
                    records.append([sym, name, ts.strftime("%d-%m-%Y %H:%M"),
                                    f"{iv/hv:4.2f}"])

        df = pd.DataFrame(records, columns=COLS)\
               .sort_values("Earnings Date")\
               .reset_index(drop=True)

        display(df if not df.empty else
                pd.DataFrame({"info":["No earnings in window."]}))
        print(f"\nOption chain **missing** for {len(non_opt)} symbols.")
        time.sleep(REFRESH)

except KeyboardInterrupt:
    clear_output()
    print("Stopped.")


[2025-05-14 12:29:44]  Earnings Dashboard (today → +7 d) – IV/HV



earnings scan:   0%|          | 0/4937 [00:00<?, ?it/s]

,info
0,No earnings in window.



Option chain **missing** for 4937 symbols.


In [2]:
# --- Mini-test : AAPL  ------------------------------------------------------
import os, logging, yfinance as yf, pandas as pd, numpy as np
from datetime import date
from math import sqrt

os.environ["YF_NO_CURL"] = "true"
os.environ["YF_USE_CURL_CFFI"] = "false"
logging.getLogger("yfinance").setLevel(logging.ERROR)

TICKER = "AAPL"
yf.Ticker("SPY").history(period="1d")              # cookie-warm-up

def next_earn(tkr: yf.Ticker):
    if hasattr(tkr, "get_next_earnings_date"):
        ts = tkr.get_next_earnings_date()
        return ts if pd.notna(ts) else None
    df = tkr.get_earnings_dates(limit=6)
    fut = df[df.index.date >= date.today()]
    return fut.index[0] if not fut.empty else None

def atm_iv(sym: str):
    try:
        t = yf.Ticker(sym)
        if not t.options:
            return None
        exp  = sorted(t.options)[0]
        oc   = t.option_chain(exp)
        spot = t.fast_info["lastPrice"]
        calls = oc.calls.assign(d=lambda d: abs(d.strike-spot))
        puts  = oc.puts .assign(d=lambda d: abs(d.strike-spot))
        civ, piv = (calls.sort_values("d").iloc[0]["impliedVolatility"],
                    puts .sort_values("d").iloc[0]["impliedVolatility"])
        if pd.isna(civ) or pd.isna(piv):
            return None
        return float((civ + piv) / 2)
    except Exception:
        return None

def hist_vol(sym: str, win=30):
    try:
        px = yf.download(sym, period=f"{win*2}d", progress=False)["Close"].tail(win+1)
        if len(px) < win+1:
            return None
        hv = np.log(px/px.shift(1)).dropna().std(ddof=0)*sqrt(252)
        return float(hv)
    except Exception:
        return None

tkr      = yf.Ticker(TICKER)
earn_ts  = next_earn(tkr)
iv       = atm_iv(TICKER)
hv       = hist_vol(TICKER)

print(f"Ticker        : {TICKER}")
print(f"Earnings date : {earn_ts if earn_ts is not None else 'N/A'}")

if iv is not None:
    print(f"30d IV        : {iv*100:5.2f} %")
else:
    print("30d IV        : N/A")

if hv is not None:
    print(f"30d HV        : {hv*100:5.2f} %")
else:
    print("30d HV        : N/A")

if iv is not None and hv not in (None, 0):
    print(f"IV/HV ratio   : {iv/hv:4.2f}")
else:
    print("IV/HV ratio   : N/A")


Ticker        : AAPL
Earnings date : 2026-04-29 06:59:00-04:00
30d IV        :  0.39 %
30d HV        : 67.49 %
IV/HV ratio   : 0.01


C:\Users\PC\AppData\Local\Temp\ipykernel_21928\2748338137.py:45: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(hv)
